https://github.com/MIT-LCP/mimic-omop/blob/master/extras/concept/chart_label_to_concept.csv
https://github.com/YerevaNN/mimic3-benchmarks/blob/master/mimic3benchmark/resources/itemid_to_variable_map.csv

In [2]:
from __future__ import absolute_import
from __future__ import print_function
import pandas as pd
import csv
import sys
import os
import numpy as np
import shutil
pd.set_option('display.max_columns', 500)

def dataframe_from_csv(path, header=0, index_col=False):
    return pd.read_csv(path, header=header, index_col=index_col)


mimic_path = "The directory including mimic-iii csv files"
path_csv = 'The directory to save the csv data'
data_path = 'The directory to save single patient records'

# Patient

In [5]:
patient = dataframe_from_csv(os.path.join(mimic_path, 'PATIENTS.csv'),index_col=False)
patient.drop(columns=['ROW_ID', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG'], inplace=True)

In [57]:
patient.groupby(['SUBJECT_ID']).head(1).shape

(46520, 3)

# ICU-Stay

In [16]:
icu = dataframe_from_csv(os.path.join(mimic_path, 'ICUSTAYS.csv'),index_col=False)
icu.drop(columns=['ROW_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'DBSOURCE'], inplace=True)

In [ ]:
icu.shape

In [ ]:
icu.groupby(['SUBJECT_ID']).head(1).shape

# Filter ICU Stays on Age

In [93]:
patient_icu = pd.merge(icu, patient, on='SUBJECT_ID')

In [ ]:
patient_icu.groupby(['SUBJECT_ID']).head(1).shape

In [ ]:
patient_icu.groupby(['HADM_ID']).head(1).shape

In [ ]:
patient_icu.shape

In [97]:
patient_icu['DOBYear'] = pd.to_datetime(patient_icu['DOB'], format= "%Y-%m-%d")
patient_icu['DOBYear'] = patient_icu.DOBYear.dt.year
patient_icu['INTIMEYear'] = pd.to_datetime(patient_icu['INTIME'], format= "%Y-%m-%d")
patient_icu['INTIMEYear'] = patient_icu.INTIMEYear.dt.year
patient_icu['AGE'] = patient_icu['INTIMEYear'] - patient_icu['DOBYear']
patient_icu.drop(columns=['DOBYear', 'INTIMEYear', 'DOB'], inplace=True)

In [98]:
patient_icu_under_1 = patient_icu[patient_icu.AGE <= 1]

In [ ]:
patient_icu_under_1.groupby(['SUBJECT_ID']).head(1).shape

In [ ]:
patient_icu_under_1.shape

In [101]:
patient_icu_adults = patient_icu[(patient_icu.AGE >= 18) & (patient_icu.AGE <= 89)]

In [ ]:
patient_icu_adults.groupby(['SUBJECT_ID']).head(1).shape

In [ ]:
patient_icu_adults.shape

In [ ]:
patient_icu = patient_icu[(patient_icu.AGE >= 18) & (patient_icu.AGE <= 89)]

# Admission

In [15]:
admission = dataframe_from_csv(os.path.join(mimic_path, 'ADMISSIONS.csv'),index_col=False)
admission.drop(columns=['ROW_ID','ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
                        'DISCHARGE_LOCATION', 'EDREGTIME', 'EDOUTTIME', 'HAS_CHARTEVENTS_DATA', 'HOSPITAL_EXPIRE_FLAG',
                        'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS'], inplace=True)

In [ ]:
admission.head()

In [ ]:
admission.shape

# Full ICUStays Information

In [18]:
adm_pat_icu = pd.merge(patient_icu, admission, on='HADM_ID')

In [19]:
adm_pat_icu.drop(columns=['SUBJECT_ID_y'], inplace=True)
col = ['SUBJECT_ID_x', 'HADM_ID', 'ICUSTAY_ID', 'GENDER', 'AGE', 'LOS', 'INTIME', 'OUTTIME', 'DIAGNOSIS','ETHNICITY']
adm_pat_icu = adm_pat_icu[col]
adm_pat_icu.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'GENDER', 'AGE', 'LOS', 'INTIME', 'OUTTIME', 
                       'DIAGNOSIS', 'ETHNICITY']

In [20]:
g_map = {'F': 1, 'M': 2}
def transform_gender(gender_series):
    global g_map
    return {'GENDER': gender_series.fillna('').apply(lambda s: g_map[s] if s in g_map else g_map[''])}

In [21]:
adm_pat_icu.update(transform_gender(adm_pat_icu.GENDER))

In [22]:
def transform_dx_into_id(df):
    df.DIAGNOSIS.fillna('nodx', inplace=True)
    dx_type = df.DIAGNOSIS.unique()
    dict_dx_key = pd.factorize(dx_type)[1]
    dict_dx_val = pd.factorize(dx_type)[0]
    dictionary  = dict(zip(dict_dx_key, dict_dx_val))
    df['DIAGNOSIS'] = df['DIAGNOSIS'].map(dictionary)
    return df

In [23]:
adm_pat_icu = transform_dx_into_id(adm_pat_icu)

In [24]:
def transform_ethn_into_id(df):
    df.ETHNICITY.fillna('nodx', inplace=True)
    dx_type = df.ETHNICITY.unique()
    dict_dx_key = pd.factorize(dx_type)[1]
    dict_dx_val = pd.factorize(dx_type)[0]
    dictionary  = dict(zip(dict_dx_key, dict_dx_val))
    df['ETHNICITY'] = df['ETHNICITY'].map(dictionary)
    return df

In [25]:
adm_pat_icu = transform_ethn_into_id(adm_pat_icu)

In [ ]:
adm_pat_icu.head()

In [ ]:
adm_pat_icu.shape

# Chart

In [ ]:
chart = dataframe_from_csv(os.path.join(mimic_path, 'CHARTEVENTS.csv'),index_col=False)
chart.drop(columns=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'STORETIME', 'CGID', 'RESULTSTATUS', 'STOPPED',
                    'WARNING', 'ERROR'], inplace=True)
chart = chart.ix[chart.ICUSTAY_ID.notnull()]
chart[['ICUSTAY_ID']] = chart[['ICUSTAY_ID']].astype(int)

# Find ICUStays with Delirium

In [ ]:
items = [228300, 228301, 228302, 228303, 228332, 228334, 228335, 228336, 228337]
delirium = chart[chart.ITEMID.isin(items)]
delirium[['ICUSTAY_ID']] = delirium[['ICUSTAY_ID']].astype(int)
icustay_delirium = delirium.ICUSTAY_ID.unique()
chart = chart[chart['ICUSTAY_ID'].isin(icustay_delirium)]

In [ ]:
delirium.VALUE.unique()

# D_ITEM - LABEL for other Tables

In [32]:
d_item = dataframe_from_csv(os.path.join(mimic_path, 'D_ITEMS.csv'),index_col=False)
d_item.drop(columns=['ABBREVIATION', 'CATEGORY', 'UNITNAME', 'CONCEPTID', 'ROW_ID', 'DBSOURCE', 'LINKSTO',
                     'PARAM_TYPE'], inplace=True) 

In [ ]:
d_item.head()

# Add Item_id to chart 

In [34]:
chart = pd.merge(chart, d_item, on='ITEMID')

In [ ]:
chart.head()

In [36]:
chart.shape

(39657694, 7)

In [37]:
chart.to_csv(os.path.join(data_path, 'chartevent.csv'),index=False)

# Load Chart with Delirium ICU_Stays

In [ ]:
chart = pd.read_csv(os.path.join(path_csv,'chartevent.csv'))

# ICU-Stay shared between admission and chart 

In [39]:
icu_adm = adm_pat_icu.ICUSTAY_ID.unique()
icu_chart = chart.ICUSTAY_ID.unique()

In [40]:
def intersection(lst1, lst2): 
    lst3 = list(set(lst1).intersection(set(lst2)))
    return lst3 

In [41]:
icustay_delirium = intersection(icu_adm, icu_chart)

In [42]:
chart = chart[chart['ICUSTAY_ID'].isin(icustay_delirium)]

In [43]:
chart.loc[chart['VALUE'] == 'No'       , 'VALUENUM'] = 0
chart.loc[chart['VALUE'] == 'Negative' , 'VALUENUM'] = 0
chart.loc[chart['VALUE'] == 'No (Stop - Not delirious)' , 'VALUENUM'] = 0
chart.loc[chart['VALUE'] == 'Yes'     , 'VALUENUM'] = 1
chart.loc[chart['VALUE'] == 'Positive', 'VALUENUM'] = 1
chart.loc[chart['VALUE'] == 'Yes (Continue)', 'VALUENUM'] = 1
chart.loc[chart['VALUE'] == 'UTA'             , 'VALUENUM'] = 2
chart.loc[chart['VALUE'] == 'Unable to Assess', 'VALUENUM'] = 2
chart.loc[chart['VALUE'] == 'Unable to Assess (Stop)', 'VALUENUM'] = 2
chart.loc[chart['VALUE'] == 'No (less than 3 errors - Stop - Not delirious)', 'VALUENUM'] = 3
chart.loc[chart['VALUE'] == 'Yes (3 or more errors, then Continue)', 'VALUENUM'] = 4

In [ ]:
chart.drop(columns=['VALUE'], inplace=True)
chart = chart.ix[chart.VALUENUM.notnull()]

In [45]:
chart.CHARTTIME = pd.to_datetime(chart.CHARTTIME)
chart.VALUEUOM = chart.VALUEUOM.fillna('').astype(str)
col = ['ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUENUM', 'VALUEUOM', 'CHARTTIME']
chart = chart[col]
chart.columns = ['ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUE', 'VALUEUOM', 'CHARTTIME']
chart[['ICUSTAY_ID']] = chart[['ICUSTAY_ID']].astype(int)

In [46]:
def check(x):
    try:
        x = float(str(x).strip())
    except:
        x = np.nan
    return x
def check_itemvalue(df):
    df['VALUE'] = df['VALUE'].apply(lambda x: check(x))
    df['VALUE'] = df.VALUE.astype(float)
    return df

In [ ]:
chart = check_itemvalue(chart)
chart = chart.ix[chart.VALUE.notnull()]

In [50]:
chart_features = [ 
       'Delirium assessment',
       'CAM-ICU Inattention',
       'CAM-ICU Altered LOC',
       'CAM-ICU MS Change',
       'CAM-ICU MS change',
       'CAM-ICU RASS LOC',
       'CAM-ICU Disorganized thinking',
       'Admission Weight (Kg)' , 'Admission Weight (lbs.)', #Weight
       'Height (cm)', 'Admit Ht', 'Height Inches', 'Length Calc Inches', 'Length in Inches', 
       'Length Calc (cm)', 'Length in cm', #Height
       'O2 saturation pulseoxymetry', 'Arterial O2 Saturation', 'SpO2', # O2 Saturation
       'Heart Rate', #Heart
       'Temperature Fahrenheit', 'Temperature F', 'Temperature F (calc)', #Temperature
       'Temperature C (calc)', 'Temperature C', 'Temperature Celsius', #Temperatur
       'BUN', 'BUN (6-20)', #BUN
       'Glucose','Glucose (serum)','Glucose finger stick','Glucose (whole blood)','Glucose (70-105)',
       'Fingerstick Glucose', #Glucose
       'Hemoglobin', # serum hemoglobin
       'Platelets', #Platelets  
       'Inspired O2 Fraction', 'FiO2', # FiO2
       'Richmond-RAS Scale', 'Goal Richmond-RAS Scale', # Sedation Scor
       'Non Invasive Blood Pressure systolic', 'NBP [Systolic]',
       'Non Invasive Blood Pressure diastolic', 'NBP [Diastolic]',
       'Non Invasive Blood Pressure mean', 'NBP Mean',
       'Arterial Blood Pressure mean', 'Arterial BP Mean', 'ART BP mean',
       'Arterial Blood Pressure systolic', 'ART BP Systolic', 'Arterial BP [Systolic]',
       'Arterial Blood Pressure diastolic', 'Arterial BP [Diastolic]', 'ART BP Diastolic',
       'Arterial O2 pressure', 'Arterial PaO2', 'PAO2', #PaO2
       'Arterial CO2 Pressure', 'Arterial PaCO2', 'pCO2', 'pCO2 (other)', 'PCO2', 'Arterial PaCO2', #PaCo2
       'Respiratory Rate', #Respiratory    
       'PH (Arterial)','Arterial pH', 'Art.pH', 'pH (Art)', #pH      
       'GCS Total',
       'GCS - Eye Opening', 'Eye Opening',
       'GCS - Motor Response', 'Motor Response',
       'GCS - Verbal Response', 'Verbal Response',
       'TSH NML(0.27-4.2)', 'TSH  (NML 0.27-4.2)', #TSH
       'Serum Osmolality',
       'Ammonia', 'ammonia', 'AMMONIA', 'AMMONIA/12-47 UMOL/L',
       'Cortisol', 'cortisol',
       'ETCO2', 'EtCO2', 'EtCO2 Clinical indication',
       'dexmedetomidine', 'Dexmedetomidine',
       'Morphine Sulfate',
       'propofol', 'Propofol (Intubation)',
       'Midazolam/Versed', 'Midazolam',
       'Lorazepam/Ativan',
       'Fentanyl', 'fentanyl mg/hr', 'Fentanyl:', 'fentanyl',
       'Cardiac Index', 'CI (PiCCO)', 'Cardiac Index (CI NICOM)', 'cardiac index o',  #CI
       'Intra Cranial Pressure', 'Intra Cranial Pressure #2' ]


In [51]:
chart = chart[chart['LABEL'].isin(chart_features)]

In [52]:
chart[chart['LABEL'] == 'Admission Weight (Kg)'].shape

(7005, 6)

In [53]:
# lbs to Kg
chart.loc[chart['LABEL'] == 'Admission Weight (lbs.)' , 'VALUE'] = chart['VALUE'] * 0.45

# Inch to CM
chart.loc[chart['ITEMID'] == 920  , 'VALUE'] = chart['VALUE'] * 2.54
chart.loc[chart['ITEMID'] == 1394 , 'VALUE'] = chart['VALUE'] * 2.54
chart.loc[chart['ITEMID'] == 4187 , 'VALUE'] = chart['VALUE'] * 2.54
chart.loc[chart['ITEMID'] == 3486 , 'VALUE'] = chart['VALUE'] * 2.54

In [54]:
# Unifying several variables into one

# Weight
chart.loc[chart['LABEL'] == 'Admission Weight (Kg)'  , 'LABEL'] = 'Weight'
chart.loc[chart['LABEL'] == 'Admission Weight (lbs.)', 'LABEL'] = 'Weight'
# Height
chart.loc[chart['LABEL'] == 'Height (cm)'       , 'LABEL'] = 'Height'
chart.loc[chart['LABEL'] == 'Admit Ht'          , 'LABEL'] = 'Height'
chart.loc[chart['LABEL'] == 'Height Inches'     , 'LABEL'] = 'Height'
chart.loc[chart['LABEL'] == 'Length Calc Inches', 'LABEL'] = 'Height'
chart.loc[chart['LABEL'] == 'Length in Inches'  , 'LABEL'] = 'Height'
chart.loc[chart['LABEL'] == 'Length Calc (cm)'  , 'LABEL'] = 'Height'
chart.loc[chart['LABEL'] == 'Length in cm'      , 'LABEL'] = 'Height' 
# O2 Saturation
chart.loc[chart['LABEL'] == 'Arterial O2 Saturation'     , 'LABEL'] = 'Oxygen Saturation'
chart.loc[chart['LABEL'] == 'O2 saturation pulseoxymetry', 'LABEL'] = 'Oxygen Saturation'
chart.loc[chart['LABEL'] == 'SpO2', 'LABEL'] = 'Oxygen Saturation'
#Temperature
chart.loc[chart['LABEL'] == 'Temperature Fahrenheit', 'LABEL'] = 'Temperature F'
chart.loc[chart['LABEL'] == 'Temperature F (calc)'  , 'LABEL'] = 'Temperature F'

chart.loc[chart['LABEL'] == 'Temperature C (calc)'  , 'LABEL'] = 'Temperature C'
chart.loc[chart['LABEL'] == 'Temperature Celsius'  , 'LABEL'] = 'Temperature C'
# BUN
chart.loc[chart['LABEL'] == 'BUN (6-20)', 'LABEL'] = 'BUN'
# Glucose
chart.loc[chart['LABEL'] == 'Glucose (serum)'      , 'LABEL'] = 'Glucose'
chart.loc[chart['LABEL'] == 'Glucose finger stick' , 'LABEL'] = 'Glucose'
chart.loc[chart['LABEL'] == 'Glucose (whole blood)', 'LABEL'] = 'Glucose'
chart.loc[chart['LABEL'] == 'Glucose (70-105)'     , 'LABEL'] = 'Glucose'
chart.loc[chart['LABEL'] == 'Fingerstick Glucose'  , 'LABEL'] = 'Glucose'
chart.loc[chart['LABEL'] == 'Non Invasive Blood Pressure systolic' , 'LABEL'] = 'NBP [Systolic]'
chart.loc[chart['LABEL'] == 'Non Invasive Blood Pressure diastolic', 'LABEL'] = 'NBP [Diastolic]'
chart.loc[chart['LABEL'] == 'Non Invasive Blood Pressure mean'     , 'LABEL'] = 'NBP Mean'
chart.loc[chart['LABEL'] == 'Arterial Blood Pressure mean', 'LABEL'] = 'ART BP mean'
chart.loc[chart['LABEL'] == 'Arterial BP Mean'            , 'LABEL'] = 'ART BP mean'
chart.loc[chart['LABEL'] == 'Arterial Blood Pressure systolic', 'LABEL'] = 'ART BP Systolic'
chart.loc[chart['LABEL'] == 'Arterial BP [Systolic]'          , 'LABEL'] = 'ART BP Systolic'
chart.loc[chart['LABEL'] == 'Arterial Blood Pressure diastolic', 'LABEL'] = 'ART BP Diastolic'
chart.loc[chart['LABEL'] == 'Arterial BP [Diastolic]'          , 'LABEL'] = 'ART BP Diastolic'
chart.loc[chart['LABEL'] == 'Resp Rate (Spont)'    , 'LABEL'] = 'Respiratory Rate (spontaneous)'
chart.loc[chart['LABEL'] == 'Respiratory Rate Set' , 'LABEL'] = 'Respiratory Rate (Set)'
chart.loc[chart['LABEL'] == 'Resp Rate (Total)'    , 'LABEL'] = 'Respiratory Rate (Total)'
chart.loc[chart['LABEL'] == 'PH (Arterial)', 'LABEL'] = 'pH'
chart.loc[chart['LABEL'] == 'Arterial pH'  , 'LABEL'] = 'pH'
chart.loc[chart['LABEL'] == 'Art.pH'       , 'LABEL'] = 'pH' 
chart.loc[chart['LABEL'] == 'pH (Art)'  , 'LABEL'] = 'pH'
chart.loc[chart['LABEL'] == 'Eye Opening'    , 'LABEL'] = 'GCS - Eye Opening'
chart.loc[chart['LABEL'] == 'Motor Response' , 'LABEL'] = 'GCS - Motor Response'
chart.loc[chart['LABEL'] == 'Verbal Response', 'LABEL'] = 'GCS - Verbal Response'
chart.loc[chart['LABEL'] == 'Pain Level'     , 'LABEL'] = 'Pain Level Response'
chart.loc[chart['LABEL'] == 'Calcium (8.4-10.2)' , 'LABEL'] = 'Calcium'
chart.loc[chart['LABEL'] == 'Magnesium (1.6-2.6)', 'LABEL'] = 'Magnesium'
chart.loc[chart['LABEL'] == 'Arterial Base Excess', 'LABEL'] = 'Base Excess'
chart.loc[chart['LABEL'] == 'Arterial CO2(Calc)', 'LABEL'] = 'Total CO2'
chart.loc[chart['LABEL'] == 'Carbon Dioxide',     'LABEL'] = 'Total CO2'  
chart.loc[chart['LABEL'] == 'Anion gap', 'LABEL'] = 'Anion Gap'
chart.loc[chart['LABEL'] == 'O2 Flow (additional cannula)', 'LABEL'] = 'O2 Flow'
chart.loc[chart['LABEL'] == 'Arterial PaCO2',        'LABEL'] = 'PaCO2'
chart.loc[chart['LABEL'] == 'Arterial CO2 Pressure', 'LABEL'] = 'PaCO2'
chart.loc[chart['LABEL'] == 'pCO2',                  'LABEL'] = 'PaCO2'
chart.loc[chart['LABEL'] == 'pCO2 (other)',          'LABEL'] = 'PaCO2'
chart.loc[chart['LABEL'] == 'PCO2',                  'LABEL'] = 'PaCO2'
chart.loc[chart['LABEL'] == 'Arterial PaCO2',        'LABEL'] = 'PaCO2'
chart.loc[chart['LABEL'] == 'Arterial PaO2',        'LABEL'] = 'PaO2' 
chart.loc[chart['LABEL'] == 'Arterial O2 pressure', 'LABEL'] = 'PaO2'
chart.loc[chart['LABEL'] == 'PAO2',                 'LABEL'] = 'PaO2'
chart.loc[chart['LABEL'] == 'Inspired O2 Fraction', 'LABEL'] = 'FiO2'
chart.loc[chart['LABEL'] == 'Richmond-RAS Scale'      , 'LABEL'] = 'Sedation Score'
chart.loc[chart['LABEL'] == 'Goal Richmond-RAS Scale' , 'LABEL'] = 'Sedation Score'
chart.loc[chart['LABEL'] == 'Cardiac Index'   , 'LABEL'] = 'CI'
chart.loc[chart['LABEL'] == 'CI (PiCCO)'      , 'LABEL'] = 'CI'
chart.loc[chart['LABEL'] == 'cardiac index o'   , 'LABEL'] = 'CI'
chart.loc[chart['LABEL'] == 'Cardiac Index (CI NICOM)'  , 'LABEL'] = 'CI'  
chart.loc[chart['LABEL'] == 'Intra Cranial Pressure'      , 'LABEL'] = 'ICP'
chart.loc[chart['LABEL'] == 'Intra Cranial Pressure #2'   , 'LABEL'] = 'ICP'
chart.loc[chart['LABEL'] == 'TSH NML(0.27-4.2)'      , 'LABEL'] = 'TSH' 
chart.loc[chart['LABEL'] == 'TSH  (NML 0.27-4.2)'    , 'LABEL'] = 'TSH'
chart.loc[chart['LABEL'] == 'Serum Osmolality'    , 'LABEL'] = 'Osmolality'
chart.loc[chart['LABEL'] == 'ammonia',               'LABEL'] = 'Ammonia' 
chart.loc[chart['LABEL'] == 'AMMONIA',               'LABEL'] = 'Ammonia'
chart.loc[chart['LABEL'] == 'AMMONIA/12-47 UMOL/L',  'LABEL'] = 'Ammonia'
chart.loc[chart['LABEL'] == 'cortisol',    'LABEL'] = 'Cortisol' 
chart.loc[chart['LABEL'] == 'EtCO2',                      'LABEL'] = 'ETCO2' 
chart.loc[chart['LABEL'] == 'EtCO2 Clinical indication',  'LABEL'] = 'ETCO2'
chart.loc[chart['LABEL'] == 'dexmedetomidine',  'LABEL'] = 'Dexmedetomidine'  
chart.loc[chart['LABEL'] == 'propofol',               'LABEL'] = 'Propofol' 
chart.loc[chart['LABEL'] == 'Propofol (Intubation)',  'LABEL'] = 'Propofol'
chart.loc[chart['LABEL'] == 'Midazolam/Versed',  'LABEL'] = 'Midazolam'  
chart.loc[chart['LABEL'] == 'Midazolam',         'LABEL'] = 'Midazolam'
chart.loc[chart['LABEL'] == 'Lorazepam/Ativan',   'LABEL'] = 'Lorazepam'
chart.loc[chart['LABEL'] == 'fentanyl mg/hr',  'LABEL'] = 'Fentanyl' 
chart.loc[chart['LABEL'] == 'fentanyl',        'LABEL'] = 'Fentanyl'
chart.loc[chart['LABEL'] == 'Fentanyl:',       'LABEL'] = 'Fentanyl'

# LAB Events

In [59]:
lab = dataframe_from_csv(os.path.join(mimic_path, 'LABEVENTS.csv'),index_col=False)
lab.drop(columns=['ROW_ID', 'VALUENUM', 'FLAG'], inplace=True)

In [60]:
d_lab = dataframe_from_csv(os.path.join(mimic_path, 'D_LABITEMS.csv'),index_col=False)
d_lab.drop(columns=['ROW_ID', 'FLUID', 'CATEGORY', 'LOINC_CODE'], inplace=True)

In [ ]:
lab_dlab = pd.merge(lab, d_lab, on='ITEMID')
lab_dlab.VALUEUOM = lab_dlab.VALUEUOM.fillna('').astype(str)
lab_dlab = lab_dlab.ix[lab_dlab.VALUE.notnull()]

In [ ]:
lab_dlab.head()

In [ ]:
lab_dlab.shape

# Add icu-stay to Lab events

In [64]:
icu_lab = pd.merge(lab_dlab, adm_pat_icu, how='right', on=['SUBJECT_ID'])

In [65]:
icu_lab.INTIME    = pd.to_datetime(icu_lab.INTIME)
icu_lab.OUTTIME   = pd.to_datetime(icu_lab.OUTTIME)
icu_lab.CHARTTIME = pd.to_datetime(icu_lab.CHARTTIME)

In [66]:
icu_lab = icu_lab[(icu_lab['CHARTTIME'] > icu_lab['INTIME']) & (icu_lab['CHARTTIME'] < icu_lab['OUTTIME'])]

In [67]:
icu_lab.drop(columns=['SUBJECT_ID', 'HADM_ID_y', 'HADM_ID_x', 'GENDER', 'AGE', 'LOS', 'INTIME', 'OUTTIME', 
                      'DIAGNOSIS', 'ETHNICITY'], inplace=True)
icu_lab[['ITEMID']] = icu_lab[['ITEMID']].astype(int)
col = ['ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUE', 'VALUEUOM', 'CHARTTIME']
icu_lab = icu_lab[col]

In [ ]:
icu_lab = check_itemvalue(icu_lab)
icu_lab = icu_lab.ix[icu_lab.VALUE.notnull()]

In [ ]:
icu_lab.head()

In [ ]:
icu_lab.shape

In [ ]:
icu_lab['LABEL'].unique()

In [ ]:
icu_lab.head()

# Filter Lab Event based on Delirium icustay  and Features

In [73]:
icu_lab = icu_lab[icu_lab['ICUSTAY_ID'].isin(icustay_delirium)]

In [ ]:
icu_lab[icu_lab['LABEL'].str.contains('Chloride')].shape

In [76]:
icu_lab_features = [
#Oxygen Saturation
        'Oxygen Saturation', 
#WBC
       'White Blood Cells', 'WBC', 'WBC Count', 
#Sodium
       'Sodium', 'Sodium, Whole Blood', 
# blood urea nitrogen (BUN)
        'Urea Nitrogen',
#Glucose   
    'Glucose', 
#Bilirubin
    'Bilirubin, Direct',# 'Bilirubin, Indirect', 'Bilirubin, Total', 
#Hemoglobin   
    'Hemoglobin',   
#Platelet
    'Platelet Count',
# Potassium
    'Potassium', 'Potassium, Whole Blood', 
# chloride
    'Chloride', 'Chloride, Whole Blood',
# Bicarbonate
       'Bicarbonate',
# Creatinine
       'Creatinine', 
# ALT
        'Alanine Aminotransferase (ALT)',
# AST
       'Asparate Aminotransferase (AST)', 
# Alkaline
       'Alkaline Phosphatase',
    'Thyroid Stimulating Hormone', #TSH
       'Osmolality, Measured', #serum osmolality
       'Carboxyhemoglobin', #Carboxyhemoglobin
       'SaO2', #Oxyhemoglobin
       'Methemoglobin', #Methemoglobin
       'Ammonia', #Ammonia
       'Cortisol', #Cortisol
       'Lactate', #Lactate
       'pH', #pH
       'pCO2', #pCO2
       'pO2' #pO2] 

In [77]:
icu_lab = icu_lab[icu_lab['LABEL'].isin(icu_lab_features)]

In [ ]:
icu_lab[icu_lab['LABEL'] == 'Hemoglobin'].shape

In [ ]:
icu_lab.shape

## Unifying variables with more than one nam

In [80]:
icu_lab.loc[icu_lab['LABEL'] == 'White Blood Cells'     , 'LABEL'] = 'WBC'
icu_lab.loc[icu_lab['LABEL'] == 'WBC Count'             , 'LABEL'] = 'WBC'
#Sodium
icu_lab.loc[icu_lab['LABEL'] == 'Sodium, Whole Blood'   , 'LABEL'] = 'Sodium'
# blood urea nitrogen (BUN)
icu_lab.loc[icu_lab['LABEL'] == 'Urea Nitrogen'   , 'LABEL'] = 'BUN'
#Bilirubin
icu_lab.loc[icu_lab['LABEL'] == 'Bilirubin, Direct'     , 'LABEL'] = 'direct bilirubin'
icu_lab.loc[icu_lab['LABEL'] == 'Hematocrit'      , 'LABEL'] = 'Hemoglobin'
#Platelet
icu_lab.loc[icu_lab['LABEL'] == 'Platelet Count'      , 'LABEL'] = 'Platelets'
# Potassium
icu_lab.loc[icu_lab['LABEL'] == 'Potassium, Whole Blood', 'LABEL'] = 'Potassium'
# chloride
icu_lab.loc[icu_lab['LABEL'] == 'Chloride, Whole Blood' , 'LABEL'] = 'Chloride'
# ALT
icu_lab.loc[icu_lab['LABEL'] == 'Alanine Aminotransferase (ALT)' , 'LABEL'] = 'ALT'
# AST
icu_lab.loc[icu_lab['LABEL'] == 'Asparate Aminotransferase (AST)', 'LABEL'] = 'AST'
# Alkaline
icu_lab.loc[icu_lab['LABEL'] == 'Alkaline Phosphatase'  , 'LABEL'] = 'Alkaline Phosphate'
icu_lab.loc[icu_lab['LABEL'] == 'Myelocytes'            , 'LABEL'] = 'Metamyelocytes'
icu_lab.loc[icu_lab['LABEL'] == 'Calcium, Total'        , 'LABEL'] = 'Calcium'
icu_lab.loc[icu_lab['LABEL'] == 'Platelet Count'        , 'LABEL'] = 'Platelets'
icu_lab.loc[icu_lab['LABEL'] == 'Red Blood Cells'       , 'LABEL'] = 'RBC'  
icu_lab.loc[icu_lab['LABEL'] == 'pCO2'                  , 'LABEL'] = 'PaCO2'
icu_lab.loc[icu_lab['LABEL'] == 'pO2'                   , 'LABEL'] = 'PaO2'
icu_lab.loc[icu_lab['LABEL'] == 'Calculated Total CO2'  , 'LABEL'] = 'Total CO2'
icu_lab.loc[icu_lab['LABEL'] == 'Basophils'             , 'LABEL'] = 'Differential-Basos'
icu_lab.loc[icu_lab['LABEL'] == 'Eosinophils'           , 'LABEL'] = 'Differential-Eos'
icu_lab.loc[icu_lab['LABEL'] == 'Lymphocytes'           , 'LABEL'] = 'Differential-Lymphs'
icu_lab.loc[icu_lab['LABEL'] == 'Monocytes'             , 'LABEL'] = 'Differential-Monos'
icu_lab.loc[icu_lab['LABEL'] == 'Osmolality, Measured'  , 'LABEL'] = 'Osmolality'
icu_lab.loc[icu_lab['LABEL'] == 'SaO2'                  , 'LABEL'] = 'Oxyhemoglobin'

icu_lab.loc[icu_lab['LABEL'] == 'Thyroid Stimulating Hormone'    , 'LABEL'] = 'TSH'

In [ ]:
icu_lab.head()

In [ ]:
icu_lab.shape

In [ ]:
icu_lab.LABEL.unique()

In [84]:
icu_lab.to_csv(os.path.join(data_path, 'icu_lab.csv'),index=False)

# Input-MV

In [85]:
input_mv = dataframe_from_csv(os.path.join(mimic_path, 'INPUTEVENTS_MV.csv'),index_col=False)
input_mv.drop(columns=['ORIGINALRATE', 'ORIGINALAMOUNT', 'COMMENTS_DATE', 'COMMENTS_CANCELEDBY', 'COMMENTS_EDITEDBY',
                       'CGID', 'RATE', 'RATEUOM', 'ORDERCATEGORYNAME', 'SECONDARYORDERCATEGORYNAME', 'LINKORDERID',
                       'ORDERCOMPONENTTYPEDESCRIPTION', 'ORDERCATEGORYDESCRIPTION', 'CONTINUEINNEXTDEPT', 'ORDERID',
                       'CANCELREASON', 'STATUSDESCRIPTION', 'ISOPENBAG', 'STORETIME', 'ENDTIME', 'ROW_ID','HADM_ID',
                       'SUBJECT_ID'], inplace=True)

In [ ]:
input_mv.head()

In [ ]:
input_mv = input_mv.ix[input_mv.ICUSTAY_ID.notnull()]
input_mv = input_mv[input_mv['ICUSTAY_ID'].isin(icustay_delirium)]
input_mv.TOTALAMOUNTUOM = input_mv.TOTALAMOUNTUOM.fillna('').astype(str)
input_mv[['ICUSTAY_ID']] = input_mv[['ICUSTAY_ID']].astype(int)
input_mv.STARTTIME = pd.to_datetime(input_mv.STARTTIME)

In [88]:
input_mv = pd.merge(input_mv, d_item, on='ITEMID')

In [89]:
input_mv = input_mv[['ICUSTAY_ID', 'ITEMID', 'LABEL', 'AMOUNT', 'AMOUNTUOM', 'STARTTIME', 'PATIENTWEIGHT']]
input_mv.columns = ['ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUE', 'VALUEUOM', 'CHARTTIME', 'PATIENTWEIGHT']

In [ ]:
input_mv.head()

In [ ]:
input_mv.shape

In [ ]:
input_mv = check_itemvalue(input_mv)
input_mv = input_mv.ix[input_mv.VALUE.notnull()]

In [93]:
input_mv_features = [
         'Dexmedetomidine (Precedex)',
         'Morphine Sulfate', 
         'Propofol',
         'Midazolam (Versed)', 'Midazolam',
         'Fentanyl', 'Fentanyl (Conc)', 'Fentanyl (Concentrate)', 'Fentanyl (Push)',
         'Lorazepam (Ativan)']

In [94]:
input_mv = input_mv[input_mv['LABEL'].isin(input_mv_features)]

In [95]:
input_mv.loc[input_mv['LABEL'] == 'Fentanyl (Conc)'          , 'LABEL'] = 'Fentanyl'
input_mv.loc[input_mv['LABEL'] == 'Fentanyl (Concentrate)'   , 'LABEL'] = 'Fentanyl'
input_mv.loc[input_mv['LABEL'] == 'Fentanyl (Push)'          , 'LABEL'] = 'Fentanyl'
input_mv.loc[input_mv['LABEL'] == 'Morphine Sulfate'  , 'LABEL'] = 'Morphine'
input_mv.loc[input_mv['LABEL'] == 'Midazolam (Versed)', 'LABEL'] = 'Midazolam' 
input_mv.loc[input_mv['LABEL'] == 'Lorazepam (Ativan)', 'LABEL'] = 'Lorazepam' 
input_mv.loc[input_mv['LABEL'] == 'Dexmedetomidine (Precedex)' , 'LABEL'] = 'Dexmedetomidine'
input_mv.loc[input_mv['LABEL'] == 'Insulin - Regular' , 'LABEL'] = 'Insulin'
input_mv.loc[input_mv['LABEL'] == 'Insulin - Humalog' , 'LABEL'] = 'Insulin'
input_mv.loc[input_mv['LABEL'] == 'Calcium Gluconate (CRRT)'   , 'LABEL'] = 'Calcium Gluconate'

# Filter ICUStay Infromation based on Delirium icustay
# Add weight from input-mv to adm-icu-pat

In [96]:
adm_pat_icu = adm_pat_icu[adm_pat_icu['ICUSTAY_ID'].isin(icustay_delirium)]

In [97]:
weight = input_mv.groupby(['ICUSTAY_ID']).head(1)[['ICUSTAY_ID', 'PATIENTWEIGHT']]

In [98]:
adm_pat_icu = pd.merge(adm_pat_icu, weight, how='left', on='ICUSTAY_ID')

In [ ]:
adm_pat_icu.head()

In [100]:
adm_pat_icu.shape

(7292, 11)

In [101]:
adm_pat_icu.ICUSTAY_ID.nunique()

7292

# Drop weight from Input_mv

In [102]:
input_mv.drop(columns=['PATIENTWEIGHT'], inplace=True)

In [ ]:
input_mv.head()

In [104]:
input_mv.shape

(156487, 6)

In [105]:
input_mv.LABEL.unique()

array(['Fentanyl', 'Propofol', 'Dexmedetomidine', 'Midazolam', 'Morphine',
       'Lorazepam'], dtype=object)

# Prescriptions¶

In [ ]:
prescription = dataframe_from_csv(os.path.join(mimic_path, 'PRESCRIPTIONS.csv'),index_col=False)
prescription.drop(columns=['ROW_ID', 'ENDDATE', 'DRUG_TYPE', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
                           'FORMULARY_DRUG_CD', 'NDC', 'ROUTE', 'FORM_UNIT_DISP', 'FORM_VAL_DISP', 'PROD_STRENGTH'],
                           inplace=True)

In [ ]:
prescription = prescription.ix[prescription.DOSE_VAL_RX.notnull()]

In [108]:
col = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'GSN', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'STARTDATE']
prescription = prescription[col]
prescription.columns = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUE', 'VALUEUOM', 'CHARTTIME']

In [109]:
prescription.loc[prescription['ICUSTAY_ID'].isnull(), 'ICUSTAY_ID'] = 123456
prescription[['ICUSTAY_ID']] = prescription[['ICUSTAY_ID']].astype(int)
prescription.CHARTTIME = pd.to_datetime(prescription.CHARTTIME)

In [110]:
icu_copy = icu.copy()
icu_copy.drop(columns=['LOS'], inplace=True)
icu_copy['INTIME'] = pd.to_datetime(icu_copy['INTIME'], format= "%Y-%m-%d")
icu_copy['INTIME'] = icu_copy.INTIME.dt.date
icu_copy['OUTTIME'] = pd.to_datetime(icu_copy['OUTTIME'], format= "%Y-%m-%d")
icu_copy['OUTTIME'] = icu_copy.OUTTIME.dt.date

In [111]:
prescription = pd.merge(prescription, icu_copy, how='left', on=['SUBJECT_ID', 'HADM_ID'])

In [112]:
prescription.CHARTTIME = pd.to_datetime(prescription.CHARTTIME)
prescription.INTIME    = pd.to_datetime(prescription.INTIME)
prescription.OUTTIME   = pd.to_datetime(prescription.OUTTIME)
prescription = prescription[(prescription['CHARTTIME'] >= prescription['INTIME']) & (prescription['CHARTTIME'] <= prescription['OUTTIME'])]

In [113]:
prescription.loc[prescription['ICUSTAY_ID_x'] == 123456, 'ICUSTAY_ID_x'] = np.nan
prescription.loc[prescription['ICUSTAY_ID_x'].isnull(), 'ICUSTAY_ID_x'] = prescription.ICUSTAY_ID_y

In [114]:
prescription.drop(columns=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID_y', 'INTIME', 'OUTTIME'], inplace=True)
prescription.columns = ['ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUE', 'VALUEUOM', 'CHARTTIME']
prescription[['ICUSTAY_ID']] = prescription[['ICUSTAY_ID']].astype(int)
prescription = prescription[prescription['ICUSTAY_ID'].isin(icustay_delirium)]

In [115]:
prescription_copy = prescription.copy()
prescription = prescription.groupby('LABEL').fillna(method='ffill')
prescription['LABEL'] = prescription_copy.LABEL

In [116]:
prescription_copy = prescription.copy()
prescription = prescription.groupby('LABEL').fillna(method='bfill')
prescription['LABEL'] = prescription_copy.LABEL

In [117]:
col = ['ICUSTAY_ID', 'ITEMID', 'LABEL', 'VALUE', 'VALUEUOM', 'CHARTTIME']
prescription = prescription[col]

In [118]:
prescription = check_itemvalue(prescription)
prescription = prescription.ix[prescription.VALUE.notnull()]

/home/ehealth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [119]:
prescription_features = [
         'Morphine Sulfate',
         'Propofol'
         'Lorazepam',
         'Midazolam'
]

In [120]:
prescription = prescription[prescription['LABEL'].isin(prescription_features)]

In [121]:
prescription.loc[prescription['LABEL'] == 'Heparin'   , 'LABEL'] = 'Heparin'
prescription.loc[prescription['LABEL'] == 'Lorazepam' , 'LABEL'] = 'Lorazepam'
prescription.loc[prescription['LABEL'] == 'Midazolam' , 'LABEL'] = 'Midazolam'
prescription.loc[prescription['LABEL'] == 'Propofol'  , 'LABEL'] = 'Propofol'
prescription.loc[prescription['LABEL'] == 'Morphine Sulfate'  , 'LABEL'] = 'Morphine'
prescription.loc[prescription['LABEL'] == 'Sodium Bicarbonate', 'LABEL'] = 'Sodium-Bicarbonate-prescrip'

In [ ]:
prescription.head()

In [123]:
prescription.shape

(13393, 6)

In [124]:
prescription.LABEL.unique()

array(['Propofol', 'Lorazepam', 'Midazolam', 'Morphine'], dtype=object)

# All Tables

In [125]:
tables = [chart, icu_lab, input_mv, prescription]
all_tables = pd.concat(tables)
all_tables[['ICUSTAY_ID']] = all_tables[['ICUSTAY_ID']].astype(int)
all_tables = all_tables.sort_values(by=['ICUSTAY_ID','CHARTTIME'], axis=0)
all_tables.reset_index(inplace=True, drop=True)

In [ ]:
all_tables = check_itemvalue(all_tables)
all_tables = all_tables.ix[all_tables.VALUE.notnull()]

In [ ]:
all_tables.head()

In [ ]:
all_tables.shape

In [ ]:
chart.ICUSTAY_ID.nunique()

In [130]:
all_tables.to_csv(os.path.join(data_path, 'all_tables.csv'),index=False)


# SAVE DATA

In [131]:
def cohort_stay_id(frame):
    cohort = frame.ICUSTAY_ID.unique()
    return cohort

# ADMISSION

In [132]:
def break_up_admission_by_unit_stay(adm, data_path, stayid, verbose=1):
    unit_stays = stayid
    nb_unit_stays = unit_stays.shape[0]
    for i, stay_id in enumerate(unit_stays):
        if verbose:
            sys.stdout.write('\rStayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dn = os.path.join(data_path, str(stay_id))
        try:
            os.makedirs(dn)  
        except:
            pass
        adm.ix[adm.ICUSTAY_ID == stay_id].to_csv(os.path.join(dn, 'admission.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [133]:
stay_id_Adm  = cohort_stay_id(adm_pat_icu)

In [ ]:
break_up_admission_by_unit_stay(adm_pat_icu, data_path, stayid=stay_id_Adm, verbose=1)

# CHART

In [135]:
def break_up_chart_by_unit_stay(chart, data_path, stayid, verbose=1):
    unit_stays = stayid
    nb_unit_stays = unit_stays.shape[0]
    for i, stay_id in enumerate(unit_stays):
        if verbose:
            sys.stdout.write('\rStayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dn = os.path.join(data_path, str(stay_id))
        try:
            os.makedirs(dn)  
        except:
            pass
        chart.ix[chart.ICUSTAY_ID == stay_id].to_csv(os.path.join(dn, 'chart.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [136]:
stay_id_chart  = cohort_stay_id(chart)

In [ ]:
break_up_chart_by_unit_stay(chart, data_path, stayid=stay_id_chart, verbose=1)

# ICU - LAB

In [ ]:
def break_up_icu_lab_by_unit_stay(icu_lab, data_path, stayid, verbose=1):
    unit_stays = stayid
    nb_unit_stays = unit_stays.shape[0]
    for i, stay_id in enumerate(unit_stays):
        if verbose:
            sys.stdout.write('\rStayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dn = os.path.join(data_path, str(stay_id))
        try:
            os.makedirs(dn)  
        except:
            pass
        icu_lab.ix[icu_lab.ICUSTAY_ID == stay_id].to_csv(os.path.join(dn, 'icu_lab.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [ ]:
stay_id_icu_lab  = cohort_stay_id(icu_lab)

In [ ]:
break_up_icu_lab_by_unit_stay(icu_lab, data_path, stayid=stay_id_icu_lab, verbose=1)

# INPUT - MV

In [ ]:
def break_up_input_mv_by_unit_stay(input_mv, data_path, stayid, verbose=1):
    unit_stays = stayid
    nb_unit_stays = unit_stays.shape[0]
    for i, stay_id in enumerate(unit_stays):
        if verbose:
            sys.stdout.write('\rStayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dn = os.path.join(data_path, str(stay_id))
        try:
            os.makedirs(dn)  
        except:
            pass
        input_mv.ix[input_mv.ICUSTAY_ID == stay_id].to_csv(os.path.join(dn, 'input_mv.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [ ]:
stay_id_input_mv  = cohort_stay_id(input_mv)

In [ ]:
break_up_input_mv_by_unit_stay(input_mv, data_path, stayid=stay_id_input_mv, verbose=1)

# PRESCRIPTION

In [ ]:
def break_up_prescription_by_unit_stay(prescription, data_path, stayid, verbose=1):
    unit_stays = stayid
    nb_unit_stays = unit_stays.shape[0]
    for i, stay_id in enumerate(unit_stays):
        if verbose:
            sys.stdout.write('\rStayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dn = os.path.join(data_path, str(stay_id))
        try:
            os.makedirs(dn)  
        except:
            pass
        prescription.ix[prescription.ICUSTAY_ID == stay_id].to_csv(os.path.join(dn, 'prescription.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [ ]:
stay_id_prescription  = cohort_stay_id(prescription)

In [ ]:
break_up_prescription_by_unit_stay(prescription, data_path, stayid=stay_id_prescription, verbose=1)

# ALL TABLES

In [ ]:
def break_up_all_tables_by_unit_stay(all_tables, data_path, stayid, verbose=1):
    unit_stays = stayid
    nb_unit_stays = unit_stays.shape[0]
    for i, stay_id in enumerate(unit_stays):
        if verbose:
            sys.stdout.write('\rStayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dn = os.path.join(data_path, str(stay_id))
        try:
            os.makedirs(dn)  
        except:
            pass
        all_tables.ix[all_tables.ICUSTAY_ID == stay_id].to_csv(os.path.join(dn, 'all_tables.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [ ]:
stay_id_all_tables  = cohort_stay_id(all_tables)

In [ ]:
break_up_all_tables_by_unit_stay(all_tables, data_path, stayid=stay_id_all_tables, verbose=1)

# Extract Time Series

In [ ]:
all_variables = list(all_tables.LABEL.unique())

In [ ]:
all_variables

In [ ]:
def filter_on_variabels(all_tables, all_variables):
    all_tables = all_tables[all_tables['LABEL'].isin(all_variables)]
    return all_tables

In [ ]:
def convert_events_to_timeseries(all_features, all_variables):
    metadata  = all_features[['CHARTTIME', 'ICUSTAY_ID']].sort_values(by=['CHARTTIME'])\
                    .drop_duplicates(keep='first').set_index('CHARTTIME')
    timeserie = all_features[['CHARTTIME', 'LABEL', 'VALUE']]\
                    .sort_values(by=['CHARTTIME'], axis=0)\
                    .drop_duplicates(subset=['CHARTTIME', 'LABEL'], keep='last')
    time_piv  = timeserie.pivot(index='CHARTTIME', columns='LABEL', values='VALUE')
    timeseries = time_piv.merge(metadata, left_index=True, right_index=True).sort_index(axis=0).reset_index()
    for v in all_variables:
        if v not in timeseries.columns:
            timeseries[v] = np.nan            
    return timeseries

In [ ]:
def binning(final, x=60):
    final.CHARTTIME = pd.to_datetime(final.CHARTTIME)
    final.INTIME = pd.to_datetime(final.INTIME)
    final['HOURS'] = (final.CHARTTIME - final.INTIME).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60
    final.drop(columns=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'INTIME', 'OUTTIME'], inplace=True)
    final['MINUTES'] = (final.HOURS).apply(lambda s: s * 60)
    final['BIN'] = (final['MINUTES']/ x).astype(int)
    final = final.fillna(final.groupby(['BIN']).transform('mean'))
    final.drop_duplicates(subset=['BIN'], keep='last',inplace=True)
    return final

In [ ]:
len(os.listdir(data_path))

In [ ]:
def extract_time_series_from_subject(data_path, all_variables):    
    for stay_dir in os.listdir(data_path):
        dn = os.path.join(data_path, stay_dir)
        try:
            sys.stdout.flush()
            
            admission  = dataframe_from_csv(os.path.join(data_path, stay_dir,'admission.csv'))
            all_tables = dataframe_from_csv(os.path.join(data_path, stay_dir, 'all_tables.csv'))
            all_tables = filter_on_variabels(all_tables, all_variables)
            all_features = all_tables.sort_values(by=['CHARTTIME'])
            timeepisode  = convert_events_to_timeseries(all_features, all_variables)
            final  = pd.merge(timeepisode, admission, on='ICUSTAY_ID')
            final  = final.sort_values(by=['CHARTTIME'])
            df_bin = binning(final, 60)
            df_bin.to_csv(os.path.join(dn, 'timeseries.csv'), index=False)
            sys.stdout.write('\rWrite StayID {0}...\n'.format(int(stay_dir)))
            if not os.path.isfile(os.path.join(dn,'timeseries.csv')):
                raise Exception
        except :
            continue
    print('DONE')

In [ ]:
extract_time_series_from_subject(data_path, all_variables)

In [161]:
import shutil
def delete_wo_timeseries(t_path):
    # import pdb;pdb.set_trace()
    for stay_dir in os.listdir(t_path):
        dn = os.path.join(t_path, stay_dir)
        try:
            stay_id = int(stay_dir)
            if not os.path.isdir(dn):
                raise Exception
        except:
            continue
        try:
            sys.stdout.flush()
            if not os.path.isfile(os.path.join(dn, 'timeseries.csv')):
                shutil.rmtree(dn)
        except:
            continue
    print('DONE deleting')

In [ ]:
delete_wo_timeseries(data_path)

In [ ]:
len(os.listdir(data_path))

In [ ]:
all_stays  = pd.Series(os.listdir(data_path))
all_filenames = []
for stay_id in (all_stays):
    df_file = os.path.join(data_path, str(stay_id), 'timeseries.csv')
    all_filenames.append(df_file)
all_series = pd.concat([pd.read_csv(f) for f in all_filenames])

In [ ]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])

combined_csv.to_csv(os.path.join(path_csv, 'all_data_delirium_mimic.csv'), index=False)
